# Huge Tensor Bug - Data loss

## Requirements
I'm running this on the standard 0.7
[RAPIDS docker containers](https://hub.docker.com/r/rapidsai/rapidsai) and also
need the following `pip` dependencies installed:

In [26]:
!pip install torch==1.0.1

If you're running this on your local machine you should have most things installed

## CODE

### Imports

In [1]:
import torch
import cudf

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

#### In order to demonstrate this bug we need a large tensor

In [4]:
#features = torch.LongTensor(15000000, 45).random_(0, 2**22)
features = torch.FloatTensor(15000000, 45).random_(-1, 1)

In [5]:
features[0]

tensor([-1.,  0.,  0., -1., -1., -1.,  0., -1., -1.,  0.,  0.,  0., -1.,  0.,
         0., -1.,  0.,  0.,  0., -1., -1., -1., -1., -1.,  0., -1., -1.,  0.,
        -1., -1., -1.,  0., -1., -1., -1.,  0.,  0., -1.,  0., -1., -1.,  0.,
         0.,  0.,  0.])

In [6]:
features[14999999]

tensor([-1.,  0.,  0., -1.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,
         0.,  0.,  0., -1.,  0., -1., -1., -1.,  0.,  0.,  0.,  0., -1., -1.,
         0.,  0., -1.,  0., -1., -1.,  0.,  0., -1.,  0., -1.,  0., -1., -1.,
         0.,  0.,  0.])

Validate that the last element in the dataframe has data

In [7]:
features2 = features.cuda()

In [8]:
features2[0]

tensor([-1.,  0.,  0., -1., -1., -1.,  0., -1., -1.,  0.,  0.,  0., -1.,  0.,
         0., -1.,  0.,  0.,  0., -1., -1., -1., -1., -1.,  0., -1., -1.,  0.,
        -1., -1., -1.,  0., -1., -1., -1.,  0.,  0., -1.,  0., -1., -1.,  0.,
         0.,  0.,  0.], device='cuda:0')

In [9]:
features2[14999999]

tensor([-1.,  0.,  0., -1.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,
         0.,  0.,  0., -1.,  0., -1., -1., -1.,  0.,  0.,  0.,  0., -1., -1.,
         0.,  0., -1.,  0., -1., -1.,  0.,  0., -1.,  0., -1.,  0., -1., -1.,
         0.,  0.,  0.], device='cuda:0')

In [10]:
features3 = features2[torch.randperm(len(features2))]

In [11]:
features3[0]

tensor([ 0.,  0., -1., -1., -1., -1.,  0.,  0.,  0., -1., -1., -1.,  0., -1.,
         0.,  0., -1., -1., -1.,  0., -1.,  0.,  0., -1.,  0., -1., -1., -1.,
        -1., -1.,  0., -1., -1.,  0., -1.,  0.,  0., -1.,  0.,  0., -1., -1.,
        -1.,  0., -1.], device='cuda:0')

In [12]:
features3[14999999]

tensor([ 0.,  0., -1.,  0., -1.,  0., -1., -1.,  0.,  0., -1.,  0., -1.,  0.,
        -1., -1.,  0.,  0., -1.,  0., -1., -1., -1.,  0.,  0.,  0.,  0.,  0.,
        -1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -1.,  0., -1.,
         0., -1., -1.], device='cuda:0')

Here's the boundary where this issue comes into play for this tensor.  All data after this point is 0.

In [13]:
features3[11930460:11930470]

tensor([[ 0., -1.,  0.,  0., -1.,  0.,  0., -1.,  0.,  0., -1., -1.,  0., -1.,
         -1.,  0.,  0., -1., -1., -1.,  0.,  0., -1.,  0., -1.,  0., -1.,  0.,
         -1., -1.,  0., -1.,  0.,  0.,  0., -1., -1.,  0.,  0., -1.,  0.,  0.,
         -1.,  0., -1.],
        [ 0., -1., -1., -1., -1.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0., -1.,  0.,  0.,  0., -1.,  0.,  0.,  0., -1., -1.,  0., -1.,
          0.,  0., -1., -1., -1., -1.,  0.,  0., -1., -1., -1., -1., -1.,  0.,
         -1.,  0.,  0.],
        [ 0.,  0., -1., -1.,  0., -1.,  0., -1., -1., -1., -1.,  0.,  0., -1.,
         -1.,  0.,  0.,  0., -1.,  0., -1.,  0., -1., -1.,  0.,  0.,  0., -1.,
         -1., -1., -1., -1.,  0., -1.,  0.,  0., -1.,  0., -1.,  0.,  0., -1.,
          0., -1., -1.],
        [-1., -1.,  0.,  0.,  0.,  0., -1., -1.,  0.,  0., -1., -1., -1., -1.,
          0., -1., -1., -1., -1., -1., -1.,  0.,  0., -1.,  0., -1.,  0.,  0.,
          0., -1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0